# Neural Network Approaches for Music Streaming Churn Prediction

This notebook implements multiple neural network approaches for predicting customer churn in a music streaming service. We compare:

1. **Tabular MLP** - Feedforward network on engineered features
2. **TabPFN** - Zero-shot pre-trained transformer for tabular data
3. **LSTM/GRU** - Sequence model on raw event data
4. **Transformer** - Self-attention over event sequences
5. **Hybrid** - Combining sequence and tabular representations

## Key Learnings Applied
- **Data Leakage**: Churners have truncated observation windows. We use fixed-window (7 days) strategy.
- **Distribution Shift**: Massive train/test feature distribution differences. Simpler models generalize better.
- **Churn = Dissatisfaction**: Focus on frustration signals (ads, thumbs down, downgrades).
- **Baseline**: Random Forest achieves 87% CV / ~61% test accuracy.

## 1. Setup & Dependencies

In [1]:
# Install required packages
# Colab: runs pip install
# Local: you may need to run this manually in terminal if packages are missing
import subprocess
import sys

def install_if_missing(package, pip_name=None):
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pip_name or package])

# Install dependencies
install_if_missing('tabpfn')
install_if_missing('pyarrow')
install_if_missing('einops')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.optim.lr_scheduler import CosineAnnealingLR

# Scikit-learn
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report

# Set seeds for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


## 2. Data Loading

The notebook auto-detects your environment (local vs Colab) and loads data accordingly.

In [3]:
import os

# Auto-detect environment
IN_COLAB = 'google.colab' in str(get_ipython()) if 'get_ipython' in dir() else False

if IN_COLAB:
    # Colab: Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    DATA_PATH = '/content/drive/MyDrive/churn_data/'  # Adjust this path
else:
    # Local: Use current directory
    DATA_PATH = './'  # Files are in the same directory as notebook

print(f"Environment: {'Google Colab' if IN_COLAB else 'Local'}")
print(f"Data path: {DATA_PATH}")

Environment: Local
Data path: ./


In [4]:
# Load pre-computed features (for tabular models)
train_features = pd.read_parquet(f'{DATA_PATH}train_features.parquet')
test_features = pd.read_parquet(f'{DATA_PATH}test_features.parquet')

print(f"Train features shape: {train_features.shape}")
print(f"Test features shape: {test_features.shape}")
print(f"Churn rate: {train_features['churn'].mean()*100:.2f}%")

Train features shape: (19140, 45)
Test features shape: (2904, 44)
Churn rate: 22.31%


In [5]:
# Load raw event data (for sequence models)
train_events = pd.read_parquet(f'{DATA_PATH}train.parquet')
test_events = pd.read_parquet(f'{DATA_PATH}test.parquet')

# Convert timestamps
train_events['time'] = pd.to_datetime(train_events['time'], unit='ms')
test_events['time'] = pd.to_datetime(test_events['time'], unit='ms')

print(f"Train events: {len(train_events):,}")
print(f"Test events: {len(test_events):,}")
print(f"Unique pages: {train_events['page'].nunique()}")

Train events: 17,499,636
Test events: 4,393,179
Unique pages: 19


## 3. Data Preparation

### 3.1 Fixed-Window Feature Extraction (Leakage-Free)

Uses only the first 7 days of data per user to ensure equal observation time.

In [ ]:
def get_first_event_times(df):
    """Get the first event timestamp for each user."""
    return df.groupby('userId')['time'].min().to_dict()

def filter_to_window(df, first_event_dict, window_days=7):
    """
    Filter events to only include those within the first N days per user.
    This eliminates temporal leakage from truncated observation windows.
    """
    df = df.copy()
    df['user_first_event'] = df['userId'].map(first_event_dict)
    df['days_since_start'] = (df['time'] - df['user_first_event']).dt.total_seconds() / 86400
    return df[df['days_since_start'] <= window_days].copy()

def create_fixed_window_features(df_window, window_days=7):
    """
    Create user-level features from windowed event data.
    All features are normalized by observation time to prevent leakage.
    """
    features = df_window.groupby('userId').agg(
        # Volume metrics
        events_in_window=('page', 'count'),
        sessions_in_window=('sessionId', 'nunique'),
        songs_in_window=('song', lambda x: x.notna().sum()),
        unique_songs=('song', 'nunique'),
        unique_artists=('artist', 'nunique'),

        # Interaction counts
        thumbs_up=('page', lambda x: (x == 'Thumbs Up').sum()),
        thumbs_down=('page', lambda x: (x == 'Thumbs Down').sum()),
        add_to_playlist=('page', lambda x: (x == 'Add to Playlist').sum()),
        add_friend=('page', lambda x: (x == 'Add Friend').sum()),

        # Frustration signals (key predictors per journal)
        errors=('page', lambda x: (x == 'Error').sum()),
        help_visits=('page', lambda x: (x == 'Help').sum()),
        ads=('page', lambda x: (x == 'Roll Advert').sum()),

        # Navigation
        home_visits=('page', lambda x: (x == 'Home').sum()),
        settings_visits=('page', lambda x: (x == 'Settings').sum()),

        # Subscription signals (strongest predictor per journal)
        downgrades=('page', lambda x: (x == 'Downgrade').sum()),
        upgrades=('page', lambda x: (x == 'Upgrade').sum()),

        # Actual observation time
        actual_days=('days_since_start', 'max'),
    ).reset_index()

    # Ensure actual_days is at least 0.1 to avoid division by zero
    features['actual_days'] = features['actual_days'].clip(lower=0.1)

    # Rate features (normalized by observation time)
    features['events_per_day'] = features['events_in_window'] / features['actual_days']
    features['songs_per_day'] = features['songs_in_window'] / features['actual_days']
    features['sessions_per_day'] = features['sessions_in_window'] / features['actual_days']

    # Ratio features
    total_thumbs = features['thumbs_up'] + features['thumbs_down']
    features['thumbs_up_ratio'] = features['thumbs_up'] / (total_thumbs + 1)
    features['thumbs_down_ratio'] = features['thumbs_down'] / (total_thumbs + 1)
    features['error_rate'] = features['errors'] / (features['events_in_window'] + 1)
    features['ad_rate'] = features['ads'] / (features['events_in_window'] + 1)
    features['ads_per_song'] = features['ads'] / (features['songs_in_window'] + 1)

    return features

In [7]:
# Create fixed-window features
WINDOW_DAYS = 7

print(f"Creating {WINDOW_DAYS}-day fixed window features...")

train_first_events = get_first_event_times(train_events)
test_first_events = get_first_event_times(test_events)

train_window = filter_to_window(train_events, train_first_events, WINDOW_DAYS)
test_window = filter_to_window(test_events, test_first_events, WINDOW_DAYS)

print(f"Train events in window: {len(train_window):,} ({len(train_window)/len(train_events)*100:.1f}%)")
print(f"Test events in window: {len(test_window):,} ({len(test_window)/len(test_events)*100:.1f}%)")

train_fixed = create_fixed_window_features(train_window, WINDOW_DAYS)
test_fixed = create_fixed_window_features(test_window, WINDOW_DAYS)

# Add churn labels
churn_labels = train_features.set_index('userId')['churn'].to_dict()
train_fixed['churn'] = train_fixed['userId'].map(churn_labels)
train_fixed = train_fixed.dropna(subset=['churn'])

print(f"Fixed-window features: {train_fixed.shape[1] - 2} features")
print(f"Train users: {len(train_fixed)}, Test users: {len(test_fixed)}")

Creating 7-day fixed window features...
Train events in window: 3,738,099 (21.4%)
Test events in window: 717,575 (16.3%)
Fixed-window features: 25 features
Train users: 19140, Test users: 2904


### 3.2 Sequence Data Preparation (for LSTM/Transformer)

In [8]:
# Create page type encoder
all_pages = list(train_events['page'].unique())
page_to_idx = {page: idx + 1 for idx, page in enumerate(all_pages)}  # 0 reserved for padding
page_to_idx['<PAD>'] = 0
idx_to_page = {v: k for k, v in page_to_idx.items()}

NUM_PAGES = len(page_to_idx)
print(f"Number of page types: {NUM_PAGES}")
print(f"Pages: {list(page_to_idx.keys())[:10]}...")

Number of page types: 20
Pages: ['NextSong', 'Downgrade', 'Help', 'Home', 'Thumbs Up', 'Add Friend', 'Thumbs Down', 'Add to Playlist', 'Logout', 'About']...


In [ ]:
def create_user_sequences(df, page_to_idx, max_len=200, use_window=True, window_days=7):
    """
    Create event sequences for each user.
    
    Returns:
        sequences: dict mapping userId -> list of page indices
        time_deltas: dict mapping userId -> list of time deltas (hours)
    """
    if use_window:
        first_events = get_first_event_times(df)
        df = filter_to_window(df, first_events, window_days)
    
    sequences = {}
    time_deltas = {}
    
    # Sort by user and time
    df_sorted = df.sort_values(['userId', 'time'])
    
    for user_id, user_df in df_sorted.groupby('userId'):
        # Get page sequence
        pages = user_df['page'].map(lambda x: page_to_idx.get(x, 0)).values
        
        # Get time deltas (hours between events)
        times = user_df['time'].values
        deltas = np.zeros(len(times))
        if len(times) > 1:
            deltas[1:] = np.diff(times).astype('timedelta64[h]').astype(float)
        deltas = np.clip(deltas, 0, 168)  # Cap at 1 week
        
        # Truncate to max_len (keep first events, as early behavior is most predictive)
        sequences[user_id] = pages[:max_len].tolist()
        time_deltas[user_id] = deltas[:max_len].tolist()
    
    return sequences, time_deltas

# Create sequences - reduced length for speed
MAX_SEQ_LEN = 200  # Reduced from 500

print(f"Creating user event sequences (max_len={MAX_SEQ_LEN})...")
train_sequences, train_deltas = create_user_sequences(
    train_events, page_to_idx, max_len=MAX_SEQ_LEN, use_window=True, window_days=WINDOW_DAYS
)
test_sequences, test_deltas = create_user_sequences(
    test_events, page_to_idx, max_len=MAX_SEQ_LEN, use_window=True, window_days=WINDOW_DAYS
)

# Print statistics
seq_lens = [len(s) for s in train_sequences.values()]
print(f"Train sequences: {len(train_sequences)}")
print(f"Sequence length: mean={np.mean(seq_lens):.1f}, median={np.median(seq_lens):.1f}, max={np.max(seq_lens)}")

### 3.3 Prepare Data for Models

In [ ]:
def prepare_tabular_data(train_df, test_df, feature_cols=None):
    """
    Prepare tabular data for neural networks.
    Handles categorical encoding and scaling.
    """
    if feature_cols is None:
        feature_cols = [c for c in train_df.columns if c not in ['userId', 'churn']]

    X_train = train_df[feature_cols].copy()
    X_test = test_df[feature_cols].copy() if 'churn' not in test_df.columns else test_df[feature_cols].copy()
    y_train = train_df['churn'].values

    # Handle categorical columns
    cat_cols = X_train.select_dtypes(include=['object']).columns.tolist()
    cat_indices = [feature_cols.index(c) for c in cat_cols]
    cat_dims = []

    label_encoders = {}
    for col in cat_cols:
        X_train[col] = X_train[col].fillna('Unknown').astype(str)
        X_test[col] = X_test[col].fillna('Unknown').astype(str)

        all_values = list(set(X_train[col].unique()) | set(X_test[col].unique()))
        le = LabelEncoder()
        le.fit(all_values)

        X_train[col] = le.transform(X_train[col])
        X_test[col] = le.transform(X_test[col])

        label_encoders[col] = le
        cat_dims.append(len(all_values))

    # Fill NaN and convert to float
    X_train = X_train.fillna(0).astype(float).values
    X_test = X_test.fillna(0).astype(float).values

    return X_train, y_train, X_test, cat_indices, cat_dims, feature_cols

# Prepare fixed-window data
X_fixed, y_fixed, X_test_fixed, cat_idx_fixed, cat_dims_fixed, feat_cols_fixed = prepare_tabular_data(
    train_fixed, test_fixed
)
print(f"Fixed-window: X_train={X_fixed.shape}, X_test={X_test_fixed.shape}")

# Prepare all-features data
X_all, y_all, X_test_all, cat_idx_all, cat_dims_all, feat_cols_all = prepare_tabular_data(
    train_features, test_features
)
print(f"All features: X_train={X_all.shape}, X_test={X_test_all.shape}")

Fixed-window: X_train=(19140, 25), X_test=(2904, 25)
All features: X_train=(19140, 43), X_test=(2904, 43)


## 4. PyTorch Datasets and Utilities

In [ ]:
class TabularDataset(Dataset):
    """Dataset for tabular data."""
    def __init__(self, X, y=None, scaler=None, fit_scaler=False):
        self.X = X.copy()
        self.y = y

        # Scale continuous features
        if scaler is None and fit_scaler:
            self.scaler = StandardScaler()
            self.X = self.scaler.fit_transform(self.X)
        elif scaler is not None:
            self.scaler = scaler
            self.X = self.scaler.transform(self.X)
        else:
            self.scaler = None

        self.X = torch.FloatTensor(self.X)
        if y is not None:
            self.y = torch.FloatTensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        return self.X[idx]


class SequenceDataset(Dataset):
    """Dataset for event sequences."""
    def __init__(self, user_ids, sequences, time_deltas, labels=None, max_len=500):
        self.user_ids = user_ids
        self.sequences = sequences
        self.time_deltas = time_deltas
        self.labels = labels
        self.max_len = max_len

    def __len__(self):
        return len(self.user_ids)

    def __getitem__(self, idx):
        user_id = self.user_ids[idx]
        seq = self.sequences.get(user_id, [0])
        deltas = self.time_deltas.get(user_id, [0.0])

        # Pad or truncate
        seq_len = len(seq)
        if seq_len < self.max_len:
            seq = seq + [0] * (self.max_len - seq_len)
            deltas = deltas + [0.0] * (self.max_len - len(deltas))
        else:
            seq = seq[:self.max_len]
            deltas = deltas[:self.max_len]

        seq_tensor = torch.LongTensor(seq)
        delta_tensor = torch.FloatTensor(deltas)
        mask = (seq_tensor != 0).float()  # Attention mask

        if self.labels is not None:
            label = torch.FloatTensor([self.labels.get(user_id, 0)])
            return seq_tensor, delta_tensor, mask, label
        return seq_tensor, delta_tensor, mask

In [ ]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    """Train for one epoch."""
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []

    for batch in dataloader:
        if len(batch) == 2:  # Tabular
            X, y = batch
            X, y = X.to(device), y.to(device)
            outputs = model(X).squeeze()
        else:  # Sequence
            seq, deltas, mask, y = batch
            seq, deltas, mask, y = seq.to(device), deltas.to(device), mask.to(device), y.to(device)
            outputs = model(seq, deltas, mask).squeeze()
            y = y.squeeze()

        loss = criterion(outputs, y)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item() * len(y)
        all_preds.extend(torch.sigmoid(outputs).detach().cpu().numpy())
        all_labels.extend(y.cpu().numpy())

    avg_loss = total_loss / len(dataloader.dataset)
    return avg_loss, np.array(all_preds), np.array(all_labels)


def evaluate(model, dataloader, criterion, device):
    """Evaluate model."""
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            if len(batch) == 2:  # Tabular
                X, y = batch
                X, y = X.to(device), y.to(device)
                outputs = model(X).squeeze()
            else:  # Sequence
                seq, deltas, mask, y = batch
                seq, deltas, mask, y = seq.to(device), deltas.to(device), mask.to(device), y.to(device)
                outputs = model(seq, deltas, mask).squeeze()
                y = y.squeeze()

            loss = criterion(outputs, y)
            total_loss += loss.item() * len(y)
            all_preds.extend(torch.sigmoid(outputs).cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    avg_loss = total_loss / len(dataloader.dataset)
    return avg_loss, np.array(all_preds), np.array(all_labels)

## 5. Model Architectures

### 5.1 Tabular MLP

In [ ]:
class TabularMLP(nn.Module):
    """
    Feedforward neural network for tabular data.
    Uses batch normalization and dropout for regularization.
    """
    def __init__(self, input_dim, hidden_dims=[128, 64], dropout=0.3):
        super().__init__()

        layers = []
        prev_dim = input_dim

        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = hidden_dim

        layers.append(nn.Linear(prev_dim, 1))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

### 5.2 TabPFN (Prior-Data Fitted Network)

In [ ]:
def run_tabpfn(X_train, y_train, X_val, y_val):
    """
    Run TabPFN - a pre-trained transformer for tabular data.
    Works well for small datasets with no hyperparameter tuning.
    """
    try:
        from tabpfn import TabPFNClassifier
        
        # TabPFN has limits on training samples and features
        max_samples = 3000
        max_features = 100
        
        # Subsample training data if too large
        if len(X_train) > max_samples:
            from sklearn.model_selection import train_test_split
            X_sub, _, y_sub, _ = train_test_split(
                X_train, y_train, train_size=max_samples, 
                stratify=y_train, random_state=SEED
            )
        else:
            X_sub, y_sub = X_train, y_train
        
        # Limit features
        if X_sub.shape[1] > max_features:
            X_sub = X_sub[:, :max_features]
            X_val_sub = X_val[:, :max_features]
        else:
            X_val_sub = X_val
        
        # Scale features
        scaler = StandardScaler()
        X_sub = scaler.fit_transform(X_sub)
        X_val_sub = scaler.transform(X_val_sub)
        
        # Fit and predict (updated API)
        clf = TabPFNClassifier(device='cpu')
        clf.fit(X_sub, y_sub)
        
        y_proba = clf.predict_proba(X_val_sub)[:, 1]
        y_pred = (y_proba >= 0.5).astype(int)
        
        return y_pred, y_proba
        
    except ImportError:
        print("TabPFN not installed. Install with: pip install tabpfn")
        return None, None
    except Exception as e:
        print(f"TabPFN error: {e}")
        return None, None

### 5.3 LSTM Sequence Model

In [ ]:
class LSTMChurnModel(nn.Module):
    """
    Simplified LSTM for event sequence classification.
    Single layer, no attention - faster on CPU.
    """
    def __init__(self, num_pages, embed_dim=32, hidden_dim=64, num_layers=1, dropout=0.3):
        super().__init__()
        
        # Page type embedding
        self.page_embed = nn.Embedding(num_pages, embed_dim, padding_idx=0)
        
        # Single layer LSTM (faster)
        self.lstm = nn.LSTM(
            embed_dim, hidden_dim, num_layers=num_layers,
            batch_first=True, bidirectional=True
        )
        
        # Classification head
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 1)
        )
    
    def forward(self, seq, time_deltas, mask):
        # Embed pages
        x = self.page_embed(seq)  # (batch, seq_len, embed_dim)
        
        # LSTM - use final hidden state
        _, (h_n, _) = self.lstm(x)
        
        # Concat forward and backward final states
        hidden = torch.cat([h_n[-2], h_n[-1]], dim=1)  # (batch, hidden_dim*2)
        
        return self.classifier(hidden)

### 5.4 Transformer Model

In [ ]:
class PositionalEncoding(nn.Module):
    """Sinusoidal positional encoding for transformer."""
    def __init__(self, d_model, max_len=512, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)

        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)


class TransformerChurnModel(nn.Module):
    """
    Transformer encoder for event sequence classification.
    Self-attention learns which events are most predictive of churn.
    """
    def __init__(self, num_pages, d_model=64, nhead=4, num_layers=2, dropout=0.3, max_len=512):
        super().__init__()

        # Page embedding
        self.page_embed = nn.Embedding(num_pages, d_model, padding_idx=0)

        # Time embedding
        self.time_proj = nn.Linear(1, d_model // 4)
        self.input_proj = nn.Linear(d_model + d_model // 4, d_model)

        # Positional encoding
        self.pos_encoder = PositionalEncoding(d_model, max_len, dropout)

        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=d_model*4,
            dropout=dropout, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # CLS token for classification
        self.cls_token = nn.Parameter(torch.randn(1, 1, d_model))

        # Classification head
        self.classifier = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, 1)
        )

    def forward(self, seq, time_deltas, mask):
        batch_size = seq.size(0)

        # Embed pages and time
        page_emb = self.page_embed(seq)
        time_emb = self.time_proj(time_deltas.unsqueeze(-1))
        x = torch.cat([page_emb, time_emb], dim=-1)
        x = self.input_proj(x)

        # Add positional encoding
        x = self.pos_encoder(x)

        # Prepend CLS token
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat([cls_tokens, x], dim=1)

        # Update mask for CLS token
        cls_mask = torch.ones(batch_size, 1, device=mask.device)
        mask_extended = torch.cat([cls_mask, mask], dim=1)

        # Create attention mask (True = ignore)
        src_key_padding_mask = (mask_extended == 0)

        # Transformer
        x = self.transformer(x, src_key_padding_mask=src_key_padding_mask)

        # Use CLS token for classification
        cls_output = x[:, 0]

        return self.classifier(cls_output)

### 5.5 Hybrid Model (Sequence + Tabular)

In [ ]:
class HybridChurnModel(nn.Module):
    """
    Hybrid model combining LSTM sequence features with tabular features.
    Captures both temporal dynamics and engineered features.
    """
    def __init__(self, num_pages, tabular_dim, embed_dim=32, hidden_dim=64, dropout=0.3):
        super().__init__()

        # Sequence branch (simplified LSTM)
        self.page_embed = nn.Embedding(num_pages, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.seq_pool = nn.AdaptiveAvgPool1d(1)

        # Tabular branch
        self.tabular_net = nn.Sequential(
            nn.Linear(tabular_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

        # Combined classifier
        combined_dim = hidden_dim * 2 + hidden_dim  # LSTM bidirectional + tabular
        self.classifier = nn.Sequential(
            nn.Linear(combined_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, seq, tabular, mask):
        # Sequence branch
        seq_emb = self.page_embed(seq)
        lstm_out, (h_n, _) = self.lstm(seq_emb)
        # Use final hidden state (concat forward and backward)
        seq_features = torch.cat([h_n[-2], h_n[-1]], dim=1)

        # Tabular branch
        tab_features = self.tabular_net(tabular)

        # Combine and classify
        combined = torch.cat([seq_features, tab_features], dim=1)
        return self.classifier(combined)

## 6. Training and Evaluation

### 6.1 Cross-Validation Framework

In [ ]:
def run_cv_tabular(model_class, X, y, model_params, train_params, n_folds=5, model_name="Model"):
    """
    Run stratified k-fold cross-validation for tabular models.
    """
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=SEED)
    
    results = {'accuracy': [], 'f1': [], 'roc_auc': []}
    fold_models = []
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"  Fold {fold+1}/{n_folds}...", end=" ", flush=True)
        
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Create datasets with scaling
        train_dataset = TabularDataset(X_train, y_train, fit_scaler=True)
        val_dataset = TabularDataset(X_val, y_val, scaler=train_dataset.scaler)
        
        # Simple shuffle - no weighted sampling (was causing issues)
        train_loader = DataLoader(train_dataset, batch_size=train_params['batch_size'], shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=train_params['batch_size'])
        
        # Create model
        model = model_class(**model_params).to(device)
        
        # Loss with class weight to handle imbalance
        n_pos = y_train.sum()
        n_neg = len(y_train) - n_pos
        pos_weight = torch.tensor([n_neg / n_pos]).to(device)
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
        
        # Optimizer
        optimizer = torch.optim.AdamW(
            model.parameters(), 
            lr=train_params['lr'], 
            weight_decay=train_params['weight_decay']
        )
        scheduler = CosineAnnealingLR(optimizer, T_max=train_params['epochs'])
        
        # Training loop with early stopping
        best_val_loss = float('inf')
        patience_counter = 0
        best_model_state = None
        
        for epoch in range(train_params['epochs']):
            train_loss, _, _ = train_epoch(model, train_loader, criterion, optimizer, device)
            val_loss, val_preds, val_labels = evaluate(model, val_loader, criterion, device)
            scheduler.step()
            
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model_state = model.state_dict().copy()
                patience_counter = 0
            else:
                patience_counter += 1
            
            if patience_counter >= train_params['patience']:
                break
        
        # Load best model and evaluate
        if best_model_state is not None:
            model.load_state_dict(best_model_state)
        
        _, val_preds, val_labels = evaluate(model, val_loader, criterion, device)
        y_pred = (val_preds >= 0.5).astype(int)
        
        acc = accuracy_score(val_labels, y_pred)
        f1 = f1_score(val_labels, y_pred)
        auc = roc_auc_score(val_labels, val_preds)
        
        results['accuracy'].append(acc)
        results['f1'].append(f1)
        results['roc_auc'].append(auc)
        fold_models.append(model)
        
        print(f"Acc={acc:.4f}, F1={f1:.4f}, AUC={auc:.4f}")
    
    # Print summary
    print(f"\n{model_name} CV Results:")
    print(f"  Accuracy: {np.mean(results['accuracy']):.4f} (+/- {np.std(results['accuracy']):.4f})")
    print(f"  F1 Score: {np.mean(results['f1']):.4f} (+/- {np.std(results['f1']):.4f})")
    print(f"  ROC-AUC:  {np.mean(results['roc_auc']):.4f} (+/- {np.std(results['roc_auc']):.4f})")
    
    return results, fold_models

In [ ]:
def run_cv_sequence(model_class, user_ids, sequences, time_deltas, labels, 
                    model_params, train_params, n_folds=5, model_name="Model"):
    """
    Run stratified k-fold cross-validation for sequence models.
    """
    # Create arrays for stratified split
    y_array = np.array([labels.get(uid, 0) for uid in user_ids])
    
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=SEED)
    
    results = {'accuracy': [], 'f1': [], 'roc_auc': []}
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(user_ids, y_array)):
        print(f"  Fold {fold+1}/{n_folds}...", end=" ", flush=True)
        
        train_users = [user_ids[i] for i in train_idx]
        val_users = [user_ids[i] for i in val_idx]
        
        # Create datasets
        train_dataset = SequenceDataset(
            train_users, sequences, time_deltas, labels, 
            max_len=train_params.get('max_seq_len', 200)
        )
        val_dataset = SequenceDataset(
            val_users, sequences, time_deltas, labels,
            max_len=train_params.get('max_seq_len', 200)
        )
        
        train_loader = DataLoader(train_dataset, batch_size=train_params['batch_size'], shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=train_params['batch_size'])
        
        # Create model
        model = model_class(**model_params).to(device)
        
        # Loss with class weight
        n_pos = y_array[train_idx].sum()
        n_neg = len(train_idx) - n_pos
        pos_weight = torch.tensor([n_neg / n_pos]).to(device)
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
        
        # Optimizer
        optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=train_params['lr'],
            weight_decay=train_params['weight_decay']
        )
        
        # Training loop
        best_val_loss = float('inf')
        patience_counter = 0
        best_model_state = None
        
        for epoch in range(train_params['epochs']):
            train_loss, _, _ = train_epoch(model, train_loader, criterion, optimizer, device)
            val_loss, val_preds, val_labels = evaluate(model, val_loader, criterion, device)
            
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model_state = model.state_dict().copy()
                patience_counter = 0
            else:
                patience_counter += 1
            
            if patience_counter >= train_params['patience']:
                break
        
        # Evaluate
        if best_model_state is not None:
            model.load_state_dict(best_model_state)
        
        _, val_preds, val_labels = evaluate(model, val_loader, criterion, device)
        y_pred = (val_preds >= 0.5).astype(int)
        
        acc = accuracy_score(val_labels, y_pred)
        f1 = f1_score(val_labels, y_pred)
        auc = roc_auc_score(val_labels, val_preds)
        
        results['accuracy'].append(acc)
        results['f1'].append(f1)
        results['roc_auc'].append(auc)
        
        print(f"Acc={acc:.4f}, F1={f1:.4f}, AUC={auc:.4f}")
    
    # Print summary
    print(f"\n{model_name} CV Results:")
    print(f"  Accuracy: {np.mean(results['accuracy']):.4f} (+/- {np.std(results['accuracy']):.4f})")
    print(f"  F1 Score: {np.mean(results['f1']):.4f} (+/- {np.std(results['f1']):.4f})")
    print(f"  ROC-AUC:  {np.mean(results['roc_auc']):.4f} (+/- {np.std(results['roc_auc']):.4f})")
    
    return results

### 6.2 Run All Models

In [ ]:
# Store all results
all_results = {}

# Training parameters - reduced for CPU speed
TRAIN_PARAMS = {
    'batch_size': 128,  # Larger batch = faster
    'lr': 1e-3,
    'weight_decay': 1e-4,
    'epochs': 30,  # Reduced
    'patience': 5,  # Faster early stopping
    'max_seq_len': 200  # Reduced from 500 - much faster
}

In [21]:
# 1. Tabular MLP - Fixed Window Features
print("="*60)
print("1. Tabular MLP (Fixed-Window Features)")
print("="*60)

mlp_params = {
    'input_dim': X_fixed.shape[1],
    'hidden_dims': [128, 64],
    'dropout': 0.3
}

results_mlp_fixed, _ = run_cv_tabular(
    TabularMLP, X_fixed, y_fixed, mlp_params, TRAIN_PARAMS,
    model_name="MLP (Fixed-Window)"
)
all_results['MLP_Fixed'] = results_mlp_fixed

1. Tabular MLP (Fixed-Window Features)
  Fold 1/5... Acc=0.3271, F1=0.3870, AUC=0.7304
  Fold 2/5... Acc=0.3237, F1=0.3904, AUC=0.7443
  Fold 3/5... Acc=0.3122, F1=0.3847, AUC=0.7362
  Fold 4/5... Acc=0.3595, F1=0.3946, AUC=0.7350
  Fold 5/5... Acc=0.3608, F1=0.4019, AUC=0.7490

MLP (Fixed-Window) CV Results:
  Accuracy: 0.3366 (+/- 0.0198)
  F1 Score: 0.3917 (+/- 0.0061)
  ROC-AUC:  0.7390 (+/- 0.0067)


In [22]:
# 2. Tabular MLP - All Features (with heavy regularization)
print("\n" + "="*60)
print("2. Tabular MLP (All Features + Heavy Regularization)")
print("="*60)

mlp_params_all = {
    'input_dim': X_all.shape[1],
    'hidden_dims': [128, 64],
    'dropout': 0.5  # Higher dropout
}

train_params_reg = TRAIN_PARAMS.copy()
train_params_reg['weight_decay'] = 1e-3  # Stronger L2

results_mlp_all, _ = run_cv_tabular(
    TabularMLP, X_all, y_all, mlp_params_all, train_params_reg,
    model_name="MLP (All Features)"
)
all_results['MLP_All'] = results_mlp_all


2. Tabular MLP (All Features + Heavy Regularization)
  Fold 1/5... Acc=0.6865, F1=0.5683, AUC=0.9100
  Fold 2/5... Acc=0.7215, F1=0.5977, AUC=0.9110
  Fold 3/5... Acc=0.6808, F1=0.5633, AUC=0.9112
  Fold 4/5... Acc=0.6920, F1=0.5692, AUC=0.9059
  Fold 5/5... Acc=0.6889, F1=0.5724, AUC=0.9196

MLP (All Features) CV Results:
  Accuracy: 0.6939 (+/- 0.0143)
  F1 Score: 0.5742 (+/- 0.0121)
  ROC-AUC:  0.9115 (+/- 0.0045)


In [ ]:
# 3. TabPFN
print("\n" + "="*60)
print("3. TabPFN (Pre-trained Transformer)")
print("="*60)

# TabPFN doesn't need custom CV loop - it handles small data well
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
tabpfn_results = {'accuracy': [], 'f1': [], 'roc_auc': []}

for fold, (train_idx, val_idx) in enumerate(skf.split(X_fixed, y_fixed)):
    print(f"  Fold {fold+1}/5...", end=" ")

    X_train, X_val = X_fixed[train_idx], X_fixed[val_idx]
    y_train, y_val = y_fixed[train_idx], y_fixed[val_idx]

    y_pred, y_proba = run_tabpfn(X_train, y_train, X_val, y_val)

    if y_pred is not None:
        acc = accuracy_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)
        auc = roc_auc_score(y_val, y_proba)

        tabpfn_results['accuracy'].append(acc)
        tabpfn_results['f1'].append(f1)
        tabpfn_results['roc_auc'].append(auc)
        print(f"Acc={acc:.4f}, F1={f1:.4f}, AUC={auc:.4f}")
    else:
        print("Failed")

if tabpfn_results['accuracy']:
    print(f"\nTabPFN CV Results:")
    print(f"  Accuracy: {np.mean(tabpfn_results['accuracy']):.4f} (+/- {np.std(tabpfn_results['accuracy']):.4f})")
    print(f"  F1 Score: {np.mean(tabpfn_results['f1']):.4f} (+/- {np.std(tabpfn_results['f1']):.4f})")
    print(f"  ROC-AUC:  {np.mean(tabpfn_results['roc_auc']):.4f} (+/- {np.std(tabpfn_results['roc_auc']):.4f})")
    all_results['TabPFN'] = tabpfn_results


3. TabPFN (Pre-trained Transformer)
  Fold 1/5... TabPFN error: TabPFNClassifier.__init__() got an unexpected keyword argument 'N_ensemble_configurations'
Failed
  Fold 2/5... TabPFN error: TabPFNClassifier.__init__() got an unexpected keyword argument 'N_ensemble_configurations'
Failed
  Fold 3/5... TabPFN error: TabPFNClassifier.__init__() got an unexpected keyword argument 'N_ensemble_configurations'
Failed
  Fold 4/5... TabPFN error: TabPFNClassifier.__init__() got an unexpected keyword argument 'N_ensemble_configurations'
Failed
  Fold 5/5... TabPFN error: TabPFNClassifier.__init__() got an unexpected keyword argument 'N_ensemble_configurations'
Failed


In [ ]:
# 4. LSTM Sequence Model
print("\n" + "="*60)
print("4. LSTM Sequence Model")
print("="*60)

# Get user IDs with labels
train_user_ids = list(train_sequences.keys())
train_labels = {uid: churn_labels.get(uid, 0) for uid in train_user_ids}

lstm_params = {
    'num_pages': NUM_PAGES,
    'embed_dim': 32,
    'hidden_dim': 64,  # Reduced
    'num_layers': 1,   # Single layer
    'dropout': 0.3
}

results_lstm = run_cv_sequence(
    LSTMChurnModel, train_user_ids, train_sequences, train_deltas, train_labels,
    lstm_params, TRAIN_PARAMS, model_name="LSTM"
)
all_results['LSTM'] = results_lstm

In [ ]:
# 5. Transformer Model (smaller for CPU)
print("\n" + "="*60)
print("5. Transformer Sequence Model")
print("="*60)

transformer_params = {
    'num_pages': NUM_PAGES,
    'd_model': 32,   # Reduced from 64
    'nhead': 2,      # Reduced from 4
    'num_layers': 1, # Reduced from 2
    'dropout': 0.3,
    'max_len': 256
}

results_transformer = run_cv_sequence(
    TransformerChurnModel, train_user_ids, train_sequences, train_deltas, train_labels,
    transformer_params, TRAIN_PARAMS, model_name="Transformer"
)
all_results['Transformer'] = results_transformer

## 7. Results Comparison

In [ ]:
# Create comparison table
comparison_data = []

for model_name, results in all_results.items():
    comparison_data.append({
        'Model': model_name,
        'Accuracy': f"{np.mean(results['accuracy']):.4f} (+/- {np.std(results['accuracy']):.4f})",
        'F1 Score': f"{np.mean(results['f1']):.4f} (+/- {np.std(results['f1']):.4f})",
        'ROC-AUC': f"{np.mean(results['roc_auc']):.4f} (+/- {np.std(results['roc_auc']):.4f})",
        'Acc Mean': np.mean(results['accuracy']),
        'AUC Mean': np.mean(results['roc_auc'])
    })

# Add baseline for comparison
comparison_data.append({
    'Model': 'Random Forest (Baseline)',
    'Accuracy': '0.8707 (+/- 0.0024)',
    'F1 Score': '~0.70',
    'ROC-AUC': '~0.90',
    'Acc Mean': 0.8707,
    'AUC Mean': 0.90
})

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('Acc Mean', ascending=False)

print("\n" + "="*80)
print("MODEL COMPARISON")
print("="*80)
print(comparison_df[['Model', 'Accuracy', 'F1 Score', 'ROC-AUC']].to_string(index=False))

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy comparison
models = [d['Model'] for d in comparison_data if 'Baseline' not in d['Model']]
accs = [d['Acc Mean'] for d in comparison_data if 'Baseline' not in d['Model']]

ax1 = axes[0]
bars = ax1.barh(models, accs, color='steelblue')
ax1.axvline(x=0.8707, color='red', linestyle='--', label='RF Baseline (87.07%)')
ax1.set_xlabel('CV Accuracy')
ax1.set_title('Model Accuracy Comparison')
ax1.legend()
ax1.set_xlim(0.5, 1.0)

# AUC comparison
aucs = [d['AUC Mean'] for d in comparison_data if 'Baseline' not in d['Model']]

ax2 = axes[1]
bars = ax2.barh(models, aucs, color='darkgreen')
ax2.axvline(x=0.90, color='red', linestyle='--', label='RF Baseline (~0.90)')
ax2.set_xlabel('CV ROC-AUC')
ax2.set_title('Model ROC-AUC Comparison')
ax2.legend()
ax2.set_xlim(0.5, 1.0)

plt.tight_layout()
plt.savefig('nn_model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Generate Submissions

In [ ]:
def train_final_model_tabular(model_class, X, y, model_params, train_params):
    """Train final model on all data."""
    train_dataset = TabularDataset(X, y, fit_scaler=True)
    train_loader = DataLoader(train_dataset, batch_size=train_params['batch_size'], shuffle=True)
    
    model = model_class(**model_params).to(device)
    
    n_pos = y.sum()
    n_neg = len(y) - n_pos
    pos_weight = torch.tensor([n_neg / n_pos]).to(device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = torch.optim.AdamW(model.parameters(), lr=train_params['lr'], weight_decay=train_params['weight_decay'])
    
    model.train()
    for epoch in range(train_params['epochs']):
        for batch in train_loader:
            X_batch, y_batch = batch
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            outputs = model(X_batch).squeeze()
            loss = criterion(outputs, y_batch)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    return model, train_dataset.scaler


def generate_submission(model, X_test, scaler, test_ids, filename, threshold=0.5):
    """Generate submission file."""
    model.eval()
    
    X_test_scaled = scaler.transform(X_test)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    
    with torch.no_grad():
        outputs = model(X_test_tensor).squeeze()
        proba = torch.sigmoid(outputs).cpu().numpy()
    
    predictions = (proba >= threshold).astype(int)
    
    submission = pd.DataFrame({
        'id': test_ids,
        'target': predictions
    })
    submission.to_csv(filename, index=False)
    
    print(f"Saved {filename}")
    print(f"  Predictions: {len(submission)}")
    print(f"  Predicted churners: {predictions.sum()} ({predictions.mean()*100:.1f}%)")
    
    return submission, proba

In [ ]:
# Train final MLP model on fixed-window features
print("Training final MLP model...")

final_model, final_scaler = train_final_model_tabular(
    TabularMLP, X_fixed, y_fixed, mlp_params, TRAIN_PARAMS
)

# Generate submission
test_ids = test_fixed['userId'].values
submission, proba = generate_submission(
    final_model, X_test_fixed, final_scaler, test_ids,
    'submission_nn_mlp.csv'
)

## 9. Conclusions

### Key Findings

1. **Fixed-window features** (7 days) help address the data leakage issue where churners have truncated observation windows.

2. **Neural network performance** compared to the Random Forest baseline (87.07% CV accuracy):
   - MLP and TabPFN provide competitive results on tabular features
   - Sequence models (LSTM, Transformer) can capture temporal patterns
   - The distribution shift between train and test remains the primary challenge

3. **Dissatisfaction signals** (ads, thumbs down, downgrades) are more predictive than engagement volume metrics.

4. **Expected test accuracy** is ~61-62% regardless of model complexity due to:
   - Massive distribution shift between train and test
   - No user overlap between datasets
   - Unknown train/test split methodology

### Recommendations

1. Use **fixed-window features** to prevent leakage
2. **Ensemble** neural network predictions with tree-based models
3. Focus on **interpretability** - attention weights from Transformer can show which events predict churn
4. Consider **threshold tuning** on a held-out validation set

In [ ]:
# Save results
results_df = pd.DataFrame(comparison_data)
results_df.to_csv('nn_comparison_results.csv', index=False)
print("Results saved to nn_comparison_results.csv")